## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-09-10-05-49-32 +0000,nyt,Nepali Troops Move to Restore Order as Death T...,https://www.nytimes.com/live/2025/09/09/world/...
1,2025-09-10-05-49-26 +0000,nyt,Fears Grow of More Violence in Nepal After Cra...,https://www.nytimes.com/2025/09/10/world/asia/...
2,2025-09-10-05-49-23 +0000,nyt,Here’s the latest.,https://www.nytimes.com/live/2025/09/10/world/...
3,2025-09-10-05-46-20 +0000,nypost,Killer Bryan Kohberger posed shirtless with wo...,https://nypost.com/2025/09/10/us-news/bryan-ko...
4,2025-09-10-05-42-02 +0000,nypost,NYPD boosting presence at ‘critical’ locations...,https://nypost.com/2025/09/10/us-news/nypd-boo...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2341/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
124,trump,53
258,new,19
445,israel,17
10,nepal,17
293,court,15


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
118,2025-09-09-22-19-57 +0000,wapo,Israeli strike targeting Hamas in Qatar ‘does ...,https://www.washingtonpost.com/world/2025/09/0...,132
332,2025-09-09-09-30-00 +0000,wsj,When a federal appeals court recently rejected...,https://www.wsj.com/politics/policy/cases-on-t...,113
41,2025-09-10-02-45-20 +0000,nypost,Court rules that Lisa Cook can keep job as Fed...,https://nypost.com/2025/09/09/us-news/court-ru...,103
84,2025-09-10-00-03-37 +0000,nypost,Trump asks EU to levy up to 100% tariffs on Ch...,https://nypost.com/2025/09/09/world-news/trump...,102
37,2025-09-10-03-01-00 +0000,wsj,A federal judge late on Tuesday blocked Presid...,https://www.wsj.com/economy/central-banking/ju...,99


## Select top headlines

In [5]:
top_rows = []
working = word_scores.copy()
remaining = latest.copy()
for _ in range(10):
    ranked_loop = remaining.assign(score=remaining['title'].apply(
        lambda t: sum(working.get(w.lower(), 0)
                      for w in re.findall(r'[A-Za-z]+', t)
                      if len(w) > 1)
    )).sort_values('score', ascending=False)
    if ranked_loop.empty:
        break
    top_story = ranked_loop.iloc[0]
    top_rows.append(top_story[['score','pubdate','source','title','link']])
    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for w in words:
        working.pop(w, None)
    remaining = remaining.drop(top_story.name)
top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
118,132,2025-09-09-22-19-57 +0000,wapo,Israeli strike targeting Hamas in Qatar ‘does ...,https://www.washingtonpost.com/world/2025/09/0...
206,79,2025-09-09-18-28-45 +0000,nypost,French President Emmanuel Macron names loyalis...,https://nypost.com/2025/09/09/world-news/frenc...
228,71,2025-09-09-17-00-25 +0000,nyt,Protests Against Nepal’s Social Media Ban Show...,https://www.nytimes.com/2025/09/09/world/asia/...
61,61,2025-09-10-01-32-56 +0000,nypost,Iryna Zarutska’s family speaks out after Ukrai...,https://nypost.com/2025/09/09/us-news/iryna-za...
75,52,2025-09-10-00-36-34 +0000,nypost,Poland’s airspace ‘repeatedly violated’ by Rus...,https://nypost.com/2025/09/09/world-news/ukrai...
332,51,2025-09-09-09-30-00 +0000,wsj,When a federal appeals court recently rejected...,https://www.wsj.com/politics/policy/cases-on-t...
162,43,2025-09-09-20-54-54 +0000,nypost,Wife of late Sen. John McCain facing calls to ...,https://nypost.com/2025/09/09/us-news/cindy-mc...
227,43,2025-09-09-17-05-42 +0000,nypost,"US test scores hit damning new lows in math, r...",https://nypost.com/2025/09/09/us-news/us-test-...
215,42,2025-09-09-17-44-10 +0000,nypost,White House fumes over Epstein birthday book s...,https://nypost.com/2025/09/09/us-news/wh-fumes...
39,41,2025-09-10-02-56-27 +0000,wapo,Judge temporarily halts firing of Fed governor...,https://www.washingtonpost.com/business/2025/0...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
